### **Multiclass Classification**
Logistic regression is one type of classification algorithm that determines, for a given set of feature values, the most likely label from a given set of labels. Binary logistic regression refers to the case that there are exactly two labels to choose from while multiclass regression is the more general case where there could be any given number of labels. The type of multiclass classification we will be implementing is built based on binary logistic regression, so we first discuss the binary case. 


**Binary Logistic Regression**

Suppose our data points have $d$ numeric features and are labeled as either $1$ or $0$. Rather than returning a label of $\pm 1$, binary logistic regression returns the probability of a data point belonging to class $1$. The predicted class is then $+1$ if the probability of beloning to $1$ is greater than $50\%$ and is $0$ if the probability of belonging to $1$ is less than $50\%$. To find the probability of a data point beloning to class $1$, the following calculation is performed, 
$$h(x) = \frac{1}{1+e^{-\langle w, x \rangle}}$$
where $x$ is the set of features and $w$ is a weights vector. 

Then if $\langle w, x \rangle > 0$, we will have $e^{-\langle w, x \rangle} < 1$, so our probability of being class $1$ is 
$$h(x) = \frac{1}{1+e^{-\langle w, x \rangle}} > \frac{1}{1+1} = \frac{1}{2}.$$
Alternatively, if $\langle w, x \rangle < 0$, we will have $e^{-\langle w, x \rangle} > 1$, so our probability of being class $1$ is 
$$h(x) = \frac{1}{1+e^{-\langle w, x \rangle}} < \frac{1}{1+1} = \frac{1}{2}.$$
In summary, if $\langle w, x \rangle > 0$, then the predicted class will be $1$. If $\langle w, x \rangle < 0$, then the predicted class will be $0$. This is then how our decision boundary is defined; the decision boundary is all vectors $x$ such that $\langle w, x \rangle = 0$. All vectors that lie on the side to which $w$ is pointing will be designated $1$ while all those that lie on the side away from which $w$ is pointing will be designated $0$. Training this algorithm therefore means determining the optimal weight vector $w$. Optimal here means the weight vector that minimizes error, so we must both quanitfy the error and find some way to minimize it. We quantify the error with Binary Cross Entropy Loss: 

$$L_s(h_w) = -\frac{1}{m}\sum_{i=1}^m(y_i\log h_w(x_i)+(1-y_i)\log(1-h_w(x_i)))$$

The next step is to find weights to minimize this. We will do that through a process called Stochastic Gradient Descent.


**Stochastic Gradient Descent**

For a given set of points with features $x_i$ and labels $y_i$, we can view the error as a function of the weight vector $w$. We want to find the $w$ that minimizes our function, and from calculus we know that the gradient of the function always points away from the minimum. We can therefore approximately locate the minimum by moving in the opposite direction of the gradient in a process known as gradient descent. Starting from some initial guess, each subsequent value is achieved by moving a set distance along the function in the opposite direction of the gradient. This set distance is the step size, and the choice of step size determines whether or not gradient descent can converge. If the step size is too small, the minimum may not have been reached after thousands of timesteps while if the step size is too big, a single step might overshoot the minimum and leave the algorithm jumping back and forth to each side of the minimum but never actually getting closer. Classical gradient descent involves using all of the training data to calculate the gradient of the loss explicitly at every step; this results in a very direct but very slow path to the minimum. Stochastic gradient descent (SGD) speeds up the process by only using a smaller subset of the data to approximate the gradient of the loss function. In SGD, the data is shuffled then split into evenly sized batches. One batch is used to calculate the gradient, then a step is taken, then the next batch is used and another step is taken, repeating until all of the batches have been used. If the algorithm still has not converged after going through all of the batches, the data is shuffled and then split into new batches and the procedure is repeated. Since we do not know a priori what the minimum is, we cannot determine convergence based on distance to the minimum. Convergence must therefore be approximated by checking how much the weights are changing with every step or checking how much the loss is changing.

Using the representation, loss, and optimizer described so far, we have a fully defined method for binary classification. We now build upon this to create a multiclass classification model. 


**One-vs-all**

One-vs-all is a method where a binary classification algorithm can be used for multiclass classification by splitting the multiclass problem into several binary problems. Assume that we have $k$ classes. To apply the one-vs-all method, we train $k$ binary logistic regression models, each of which predicts the probability of being in one of the specific $k$ classes. Once this has been done, a the final predicted class of each data point is determined by labeling it with the class that it had the highest probability of belonging to. Implementing this algorithm would look something like the steps outlined below. 

$\text{inputs:}$

$S = \text{ the training set comprised of $m$ labeled sets of features}(\vec{x}_i, y_i)$

$A = \text{ the binary classification algorithm in use, ex. binary logistic regression}$

$\text{For each } k \in \text{ the label set } \mathcal{Y}:$

$\quad \text{Define } S_k \text{ to be the set of all examples but where the label is 1 if the example has label $k$ and 0 otherwise}$

$\quad \text{Define } h_k = A(S_k) \text{ as a binary predictor that predicts 1 if } x \in S_k$

$\text{Output:}$

$\quad \text{Define the multiclass predictor by }h(\vec{x}) = \text{argmax}_{k \in \mathcal{Y}} h_i(\vec{x})$

In summary, the multiclass predictor is composed of one binary predictor for each class that simply predicts whether or not the data belongs to that class. These prections are then gathered and the final label is the class that $\vec{x}$ had the highest probability of being in. 

**One-vs-one**

The other multiclass classifier we will be examining is the one-vs-one classifier. Like the one-vs-all algorithm, the one-vs-one algorithm builds a multiclass predictor out of several binary predictors. For this predictor, for each pair $i,j$ of classes, construct a subset $S_{i,j}$ of the training data $S$ that contains only the examples that are in class $i$ or $j$. Then for each subset $S_{i,j}$, set the label of $\vec{x}$ to $1$ if $\vec{x}$ is in class $i$ and -1 if $x$ is in class $j$. A binary classifier $h_{i,j}$ is defined for each $S_{i,j}$. A binary classifier $h_{i,j}$ is defined for each $S_{i,j}$. To get the final prediction for the class of $\vec{x}$, $\vec{x}$ is fed into each of the binary classifiers $h_{i,j}$. The predicted class of $\vec{x}$ is chosen to be the class that was predicted most frequently across all of the binary classifiers. Implementing this algorithm would look something like the steps outlined below. 

$\text{inputs:}$

$S = \text{ the training set comprised of $m$ labeled sets of features}(\vec{x}_i, y_i)$

$A = \text{ the binary classification algorithm in use, ex. binary logistic regression}$

$\text{For each } i,j \in \text{ the label set } \mathcal{Y} \text{ with } i < j:$

$\quad \text{Construct } S_{i,j} \text{ by adding each $\vec{x}_k$ such that } y_k \in \{i,j\}$

$\quad \text{Define } h_{i,j} = A(S_{i,j}) \text{ as a binary predictor that predicts either $i$ (+1) or $j$ (-1) for each input }$

$\text{Output:}$

$\quad \text{Define the multiclass predictor by }h(\vec{x}) = \text{argmax}_{i \in \mathcal{Y}} \left( \sum_{j \in \mathcal{Y}} \text{sign}(j-i)H_{i,j}(\vec{x} \right)$

**Implementation**

Now that we have established the theoretical basis for these algorithms, we will implement each of them below along with several test cases to demonstrate that they are performing as expected.

In [1]:
print('code here')

code here
